In [100]:
pip install scikit-learn

  Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
  Using cached scipy-1.12.0-cp39-cp39-win_amd64.whl (46.2 MB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'd:\yahel\phd\courses\ML in biology research\final_project\final_project_ml\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import os
os.chdir('D:\yahel\phd\courses\ML in biology research\\final_project')
os.getcwd()

'd:\\yahel\\phd\\courses\\ML in biology research\\final_project'

In [2]:

import numpy as np
import pandas as pd
import random
import string
import pickle
import copy
import glob

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedGroupKFold , train_test_split

random.seed(42)
mpl.rcParams['figure.facecolor'] = 'white'
%matplotlib inline

C:\Users\yahelc\AppData\Local\Temp\ipykernel_14516\4074392612.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [34]:
# function to calculate ratios
# object must be pandas series
def calculate_ratio(nome: pd.Series, deno: pd.Series = None):
    """
    function to calculate ratios
    :param nome:
    :param deno:
    :return:
    """
    
    if deno is None:
        deno = nome
        
    return(nome.value_counts() / deno.value_counts().sum())

In [3]:
# read database #
path_to_data = os.path.join('data', 'diabetic_data.csv')
whole_data_df = pd.read_csv(path_to_data)
whole_data_df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [ ]:
#  Look at the big picture (Quick look at the data structure)
possibale_val_list = [whole_data_df[x].unique() for x in whole_data_df]
possibale_val_list = pd.DataFrame(zip(list(whole_data_df.columns),possibale_val_list))

#### Filter dataset for diabetic pateints diagnosed at least once 

In [ ]:
# change the unknown valuse in the data to np.nan
whole_data_df[whole_data_df == "?"] = np.nan
whole_data_df[whole_data_df == "None"] = np.nan
whole_data_df[whole_data_df == "Unknown/Invalid"] = np.nan

# how many var and obs in data, data types and non -missing values
mis = whole_data_df.isna().sum()
whole_data_df.info()

# drop all fitures with only one value from the data set
for fiture in whole_data_df[whole_data_df.columns[2:]]:
    if len(set(whole_data_df[fiture])) == 1:
        whole_data_df.pop(fiture)
whole_data_df.head()

In [ ]:
# drop all fitures with only one value from the data set
for fiture in whole_data_df[whole_data_df.columns[2:]]:
    if len(set(whole_data_df[fiture])) == 1:
        whole_data_df.pop(fiture)
whole_data_df.head()

In [4]:
diagnosis_cols_list = ['diag_' + str(num) for num in [1,2,3]]

# Regex pattern to locate diabeteic pateints 
pattern = r'250(\.\d{2})?'

# count diabetes diagnosis appearance  in all 3 daignosis columns 
diag_count = whole_data_df[diagnosis_cols_list].apply(lambda x: x.str.contains(pattern)).sum(axis =1)

diabetic_only_df = whole_data_df[diag_count > 0]
print('%d encounters out of %d total encounters are of diabetic pateints \n i.e. ~ %.2f%% '\
    %(diabetic_only_df.shape[0],whole_data_df.shape[0],
      (diabetic_only_df.shape[0]/whole_data_df.shape[0])*100))

38024 encounters out of 101766 total encounters are of diabetic pateints 
 i.e. ~ 37.36% 


C:\Users\yahelc\AppData\Local\Temp\ipykernel_14516\668848931.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  diag_count = whole_data_df[diagnosis_cols_list].apply(lambda x: x.str.contains(pattern)).sum(axis =1)


### determine repeated encounter by the same patient

In [5]:
duplicated_encounters = sum(diabetic_only_df['patient_nbr'].duplicated())
print('%d encounters are patient repeated ones, ~%.2f%%' % \
    (duplicated_encounters,
    (duplicated_encounters/diabetic_only_df.shape[0]*100)))


print(diabetic_only_df['patient_nbr'].value_counts().value_counts())


6836 encounters are patient repeated ones, ~17.98%
count
1     26849
2      3091
3       750
4       242
5       114
6        55
7        31
8        14
10       10
11        9
9         7
15        4
12        2
14        2
17        2
28        1
13        1
18        1
19        1
22        1
38        1
Name: count, dtype: int64


In [23]:
calculate_ratio(diabetic_only_df['readmitted'])

In [36]:
# an imbalanced label! 

# get the repeated patients readmission labels 
duplicated_df = diabetic_only_df[diabetic_only_df['patient_nbr'].duplicated()]
duplicated_df = duplicated_df[~duplicated_df['patient_nbr'].duplicated()]

# the precenatge of 
print('\n\nprecenatge of label classes beloning to repeated patients \n',duplicated_df['readmitted'].value_counts()/diabetic_only_df['readmitted'].value_counts())

precentage of classes of the labels
 readmitted
NO     0.547102
>30    0.343362
<30    0.109536
Name: count, dtype: float64


precenatge of label classes beloning to repeated patients 
 readmitted
<30    0.163745
>30    0.147212
NO     0.083401
Name: count, dtype: float64


## Since a large portion of the encounters is a duplication of the same patient 
## We decided to treat each encounter as a different patient
## Therefore for test-train split we will follow the group_stratified split 

In [55]:
# stratify by pateints' ID as group 
groups_for_stratification = diabetic_only_df['patient_nbr']
X = diabetic_only_df.drop('readmitted',axis=1)
Y = diabetic_only_df['readmitted']

# instentiate the StratifiedGroupKFold class
sgkf = StratifiedGroupKFold(n_splits=5) 

# use the CV split to create 5 spilt of a 80-20 ratio
cv_division = sgkf.split(X, Y,groups_for_stratification)

# unpack the first one - randomly 
train_test_index_array, *_ = cv_division

train = diabetic_only_df.iloc[train_test_index_array[0]]
test = diabetic_only_df.iloc[train_test_index_array[1]]

display(pd.DataFrame({'Complete dataset' : calculate_ratio(diabetic_only_df['readmitted']),
                      'Train datset (80%)' : calculate_ratio(train['readmitted']),
                      'Test datset (20%)' : calculate_ratio(test['readmitted'])}))

print('There are %d pateint with overlap between train-test cohorts' % sum(np.in1d(train['patient_nbr'].unique(), test['patient_nbr'].unique())))

if len(glob.glob('D:/yahel/phd/courses/ML in biology research/final_project/data/data_split*.csv')) == 0:
    train.to_csv(os.path.join(os.getcwd(),'data','data_split','train_cohort.csv'))
    test.to_csv(os.path.join(os.getcwd(),'data','data_split','test_cohort.csv'))

,Complete dataset,Train datset (80%),Test datset (20%)
readmitted,,,
NO,0.547102,0.547107,0.547080
>30,0.343362,0.343360,0.343372
<30,0.109536,0.109533,0.109548


There are 0 pateint with overlap between train-test cohorts
